In [1]:
import sys
sys.path.append('/Users/ekang/anaconda2/lib/python2.7/site-packages')

In [2]:
import os, cv2, random
import numpy as np
import pandas as pd

from glob import iglob
from keras.applications import VGG16, resnet50, xception
from keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns
%matplotlib inline 

from keras.models import Sequential, Model
from keras.layers import Input, Dropout, Flatten, Convolution2D, MaxPooling2D
from keras.layers import ZeroPadding2D, Dense, Activation, BatchNormalization, GlobalAveragePooling2D
from keras.optimizers import RMSprop, Adam
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras import backend as K

from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

Using Theano backend.


# Arguments for models

### VGG16
    # Arguments
        include_top: whether to include the 3 fully-connected
            layers at the top of the network.
        weights: one of `None` (random initialization)
            or "imagenet" (pre-training on ImageNet).
        input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False (otherwise the input shape
            has to be `(224, 224, 3)` (with `tf` dim ordering)
            or `(3, 224, 244)` (with `th` dim ordering).
            It should have exactly 3 inputs channels,
            and width and height should be no smaller than 48.
            E.g. `(200, 200, 3)` would be one valid value.
        classes: optional number of classes to classify images
            into, only to be specified if `include_top` is True, and
            if no `weights` argument is specified.
            
### resnet50
    # Arguments
        include_top: whether to include the 3 fully-connected
            layers at the top of the network.
        weights: one of `None` (random initialization)
            or "imagenet" (pre-training on ImageNet).
        input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False (otherwise the input shape
            has to be `(224, 224, 3)` (with `tf` dim ordering)
            or `(3, 224, 244)` (with `th` dim ordering).
            It should have exactly 3 inputs channels,
            and width and height should be no smaller than 197.
            E.g. `(200, 200, 3)` would be one valid value.
        classes: optional number of classes to classify images
            into, only to be specified if `include_top` is True, and
            if no `weights` argument is specified.

### xception
    # Arguments
        include_top: whether to include the fully-connected
            layer at the top of the network.
        weights: one of `None` (random initialization)
            or "imagenet" (pre-training on ImageNet).
        input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False (otherwise the input shape
            has to be `(299, 299, 3)`.
            It should have exactly 3 inputs channels,
            and width and height should be no smaller than 71.
            E.g. `(150, 150, 3)` would be one valid value.
        classes: optional number of classes to classify images
            into, only to be specified if `include_top` is True, and
            if no `weights` argument is specified.

## Import Image to (224, 224, 3) for VGG16, and ResNet50

In [3]:
TRAIN_DIR = 'data/train/'
TEST_DIR = 'data/test_stg1/'
FISH_CLASSES = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']
NB_CLASSES = len(FISH_CLASSES)
ROWS = 224  #720
COLS = 224 #1280
CHANNELS = 3

def get_images(fish):
    """Load files from train folder"""
    fish_dir = TRAIN_DIR+'{}'.format(fish)
    images = [fish+'/'+im for im in os.listdir(fish_dir)]
    return images

def read_image(src):
    """Read and resize individual images"""
    # cv2.IMREAD_COLOR : Loads a color image. Any transparency of image will be neglected. It is the default flag.
    im = cv2.imread(src, cv2.IMREAD_COLOR)
    # bicubic interpolation is often chosen over bilinear interpolation or nearest neighbor in image resampling
    im = cv2.resize(im, (COLS, ROWS), interpolation=cv2.INTER_CUBIC)
    
    return im

In [ ]:
files = []
y_all = []

for fish in FISH_CLASSES:
    fish_files = get_images(fish)
    
    # Add elements to empty list
    files.extend(fish_files)
    
    # repeat fish, len(fish_files) times
    y_fish = np.tile(fish, len(fish_files))
    y_all.extend(y_fish)
    
    print("{0} photos of {1}".format(len(fish_files), fish))
    
y_all = np.array(y_all)

1719 photos of ALB
200 photos of BET
117 photos of DOL
67 photos of LAG
465 photos of NoF
299 photos of OTHER
176 photos of SHARK
734 photos of YFT


## VGG16 Model

### 1. Prepare Data to Correct Input
### 2. Model

In [ ]:
def vgg16_preprocess_from_img_path(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    
    return x

# uint8	Unsigned integer (0 to 255)
X_all = np.ndarray((len(files), CHANNELS, ROWS, COLS), dtype=np.uint8)

for i, im in enumerate(files):
    X_all[i] = vgg16_preprocess_from_img_path(TRAIN_DIR+im)
    if i%1000 == 0: print('Processed {} of {}'.format(i, len(files)))

print(X_all.shape)

Processed 0 of 3777
Processed 1000 of 3777
Processed 2000 of 3777
Processed 3000 of 3777

In [ ]:
# One Hot Encoding Labels
# Categorical to Numerical
y_all = LabelEncoder().fit_transform(y_all)
y_all = np_utils.to_categorical(y_all)

X_train, X_valid, y_train, y_valid = train_test_split(X_all, y_all, 
                                                    test_size=0.2, random_state=23, 
                                                    stratify=y_all)

In [ ]:
#Get back the convolutional part of a VGG network trained on ImageNet
model_vgg16_conv = VGG16(weights='imagenet', include_top=False)
# model_vgg16_conv.summary()

#Create your own input format (here 3x200x200)
input_shape = Input(shape=(CHANNELS,ROWS,COLS),name = 'image_input')

#Use the generated model 
output_vgg16_conv = model_vgg16_conv(input_shape)

#Add the fully-connected layers with BatchNormalization and Dropout
x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(4096, activation='relu', name='fc1')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(NB_CLASSES, activation='softmax', name='predictions')(x)

#Create your own model 
vgg16_model = Model(input=input_shape, output=x)

In [ ]:
# # first: train only the top layers (which were randomly initialized)
# # i.e. freeze all convolutional InceptionV3 layers
# for layer in model_vgg16_conv.layers:
#     layer.trainable = True

In [ ]:
# compile the model (should be done *after* setting layers to non-trainable)
vgg16_model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [ ]:
import time
start_time = time.time()

# checkpoint
filepath = "Vgg16 model and weights/weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

# Fit the model
history = vgg16_model.fit(X_train, y_train, batch_size=64, nb_epoch=50, validation_split=0.2, verbose=1, shuffle=True)

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
%matplotlib inline
# summarize history for accuracy
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()